## **UNION**

#### ПРИНЦИП И УСЛОВИЯ РАБОТЫ

!!!ORDER BY и LIMIT относятся ко всей получившейся таблице

!!!все кроме union убирают дубликаты

```sql
SELECT         n columns
FROM 
         table_1
UNION ALL
SELECT 
         n columns
FROM 
         table_2
...
UNION ALL
SELECT 
         n columns
FROM 
         table_n
```

*Обратимся к нашему датасету о транспортной компании и посмотрим, как сформировать справочник с ID всех таблиц и указанием объекта, к которому он относится.*
```sql
SELECT
         c.city_id object_name, 'id города' object_type
FROM 
         sql.city c
UNION ALL
SELECT
         d.driver_id other_name, 'id водителя' other_type
FROM 
         sql.driver d
UNION ALL
SELECT
         s.ship_id, 'id доставки'
FROM 
         sql.shipment s
UNION ALL
SELECT
         c.cust_id, 'id клиента'
FROM 
         sql.customer c
UNION ALL
SELECT
         t.truck_id, 'id грузовика'
FROM 
         sql.truck t
ORDER BY 1
```

*отсортируем каждую часть запроса по отдельности и объединим результаты*
```sql
(SELECT book_name object_name, 'книга' object_descritption 
FROM public.books
ORDER BY 1
LIMIT 1)
UNION ALL
(SELECT movie_title, 'фильм' 
FROM sql.kinopoisk
ORDER BY 1
LIMIT 1)
```

Для типизации в Postgres составляется запрос по модели column_name::column_type.
Таким образом, чтобы перевести city_id в текст, нам потребуется написать city_id::text.

*мы хотим вывести список всех id городов и их названий в одном столбце*
```sql
SELECT 
         c.city_id::text
FROM
         sql.city c
UNION ALL
SELECT 
         cc.city_name
FROM
         sql.city cc
```

*Напишите запрос, который объединит в себе все почтовые индексы водителей и их телефоны в единый столбец-справочник contact. Также добавьте столбец с именем водителя first_name и столбец contact_type с типом контакта (phone или zip в зависимости от типа). Отсортируйте список по столбцу с контактными данными в порядке возрастания, а затем — по имени водителя.*
```sql
SELECT 
        d.zip_code::text contact,
        d.first_name,
        'zip' contact_type
FROM
        sql.driver d
UNION ALL
SELECT 
        d.phone,
        d.first_name,
        'phone'
FROM
        sql.driver d
ORDER BY 1, 2
```

#### **группировка**

Напишите запрос, который выводит общее число доставок total_shipments, а также количество доставок в каждый день. Необходимые столбцы: date_period, cnt_shipment. Не забывайте о единой типизации. Упорядочите по убыванию столбца date_period.*
```sql
SELECT
        s.ship_date::text date_period,
        COUNT(s.ship_id)
FROM
        sql.shipment s
GROUP BY s.ship_date
UNION ALL
SELECT
        'total_shipments',
        COUNT(s.ship_id)
FROM
        sql.shipment s
ORDER BY 1 DESC
```

#### **фильтрация**

*Напишите запрос, который выведет все города и штаты, в которых они расположены, а также информацию о том, была ли осуществлена доставка в этот город:*
+ *если в город была осуществлена доставка, то выводим 'доставка осуществлялась';*
+ *если нет — выводим 'доставка не осуществлялась'.*

*Столбцы к выводу: city_name, state, shipping_status. Отсортируйте в алфавитном порядке по городу, а затем — по штату.*
```sql
SELECT
        DISTINCT c.city_name city_name,
        c.state state,
        'доставка осуществлялась' shipping_status
FROM
        sql.city c
        LEFT JOIN sql.shipment s on c.city_id = s.city_id
WHERE s.ship_id IS NOT NULL
UNION ALL
SELECT
        DISTINCT c.city_name city_name,
        c.state state,
        'доставка не осуществлялась' shipping_status
FROM
        sql.city c
        LEFT JOIN sql.shipment s on c.city_id = s.city_id
WHERE  s.ship_id IS NULL
ORDER BY 1, 2
```

*Напишите запрос, который выводит два столбца: city_name и shippings_fake. Выведите города, куда совершались доставки. Пусть первый столбец содержит название города, а второй формируется так:*

+ *если в городе было более десяти доставок, вывести количество доставок в этот город как есть;*
+ *иначе — вывести количество доставок, увеличенное на пять.*
*Отсортируйте по убыванию получившегося «нечестного» количества доставок, а затем — по имени в алфавитном порядке.*
```sql
SELECT
        c.city_name city_name,
        COUNT(s.ship_id) shippings_fake
FROM
        sql.city c
        JOIN sql.shipment s on c.city_id = s.city_id
GROUP BY c.city_name
HAVING COUNT(s.ship_id) > 10
UNION ALL
SELECT
        c.city_name city_name,
        COUNT(s.ship_id)+5 shippings_fake
FROM
        sql.city c
        JOIN sql.shipment s on c.city_id = s.city_id
GROUP BY c.city_name
HAVING COUNT(s.ship_id) <= 10
ORDER BY 2 DESC, 1
```

*Как бы вы рассчитали максимальную оценку в каждом классе?*

```sql
SELECT 
  CLASS, 
  MAX(GRADE) INTO TABLE_B 
FROM 
  TABLE_A 
GROUP BY 
  CLASS
```
*Этот запрос группирует данные по каждому классу, а затем находит максимальное значение. Он дополнительно сохраняет результаты в новую таблицу (TABLE_B), чтобы их можно было запросить позже.*

*Как найти не только самую высокую оценку в каждом классе и учащегося, получившего её?*
```sql
SELECT 
  a.CLASS, 
  a.GRADE, 
  a.STUDENT 
FROM 
  TABLE_A a 
  INNER JOIN TABLE_B b ON a.CLASS = b.CLASS 
  AND a.GRADE = b.GRADE
```
*Этот запрос выбирает всех учащихся и их оценки из исходной таблицы TABLE_A, где указаны классы и оценки, которые отображаются в таблице максимальных значений TABLE_B. Внутреннее соединение работает как фильтр, сохраняя только максимальные комбинации классов/оценок, потому что только в этом случае оценка появляется в таблице TABLE_B.*


### **EXCEPT** *(исключить(кроме B,вместо union))*
```sql
SELECT 
         n columns
FROM 
         table_1
EXCEPT
SELECT 
         n columns
FROM 
         table_2
```

*Предположим, нам нужно узнать, в какие города осуществлялась доставка, за исключением тех, в которых проживают водители.*
```sql
SELECT
         c.city_name
FROM
         sql.shipment s
JOIN sql.city c ON s.city_id = c.city_id
EXCEPT
SELECT
         cc.city_name
FROM
         sql.driver d 
JOIN sql.city cc ON d.city_id=cc.city_id
ORDER BY 1
```

### **INTERSECT** *(пересечение(и там и там, общее))*
```sql
SELECT 
         n columns
FROM 
         table_1
INTERSECT
SELECT 
         n columns
FROM 
         table_2
```

*Предположим, нам надо вывести совпадающие по названию города и штаты.*
```sql
SELECT          c.city_name object_name
FROM          sql.city c
INTERSECT
SELECT 
         cc.state
FROM          sql.city cc
ORDER BY 1
```

```sql
SELECT          N columns
FROM          table_1
UNION / UNION ALL / EXCEPT / INTERSECT 
SELECT          N columns
FROM          table_2
```

Есть три варианта записи условия по году:

1) Записать через диапазон  и строковые литералы `ship_date between '01-01-2017' and '12-31-2017'` или `ship_date between 'Jan 1, 2017' AND 'Dec 31, 2017'`

2) С помощью функции EXTRACT извлечь год из даты, тогда результат будет в числовом формате и его можно сравнить со значением 2017 `extract(YEAR from ship_date) = 2017`

3) Привести дату к типу строка и воспользоваться поиском по шаблону `ship_date::text like '2017%'`